We're going to try to incorporate weather data into the analysis, with the hypothesis that precipitation and extreme temperatures will have negative effects on enforcement, so we might want to control for weather.

The weather data was downloaded several years at a time from the NOAA weather export tool. It was chunked because the data was too big to download in one pass. Here we combine it back together, and perform some basic quality checks.

In [1]:
import pandas as pd
import os

dfs = []
for f in os.scandir('../data/weather'):
    dfs.append(pd.read_csv(f.path))

In [2]:
len(dfs)

6

In [4]:
[len(df.columns) for df in dfs]

[13, 13, 13, 13, 13, 13]

In [5]:
df = pd.concat(dfs)

In [7]:
[d.shape[0] for d in dfs]

[10253, 79660, 98762, 106312, 64811, 77841]

In [8]:
sum([d.shape[0] for d in dfs])

437639

In [6]:
df.shape[0]

437639

In [12]:
df.sort_values(by=['DATE', 'STATION'], inplace=True)

In [13]:
df.head()

,DATE,PRCP,SNOW,SNWD,STATION,TAVG,TMAX,TMIN,WT01,WT03,WT04,WT05,WT06,WT11
328,20000101,0.00,0.0,NaN,GHCND:USC00050848,-9999.0,54,29,-9999,-9999,-9999,-9999,-9999,-9999
2949,20000101,0.06,1.3,NaN,GHCND:USC00051681,-9999.0,40,18,-9999,-9999,-9999,-9999,-9999,-9999
4608,20000101,0.00,0.0,NaN,GHCND:USC00053629,-9999.0,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999
6801,20000101,0.00,0.0,NaN,GHCND:USC00055116,-9999.0,57,21,-9999,-9999,-9999,-9999,-9999,-9999
8584,20000101,0.00,0.0,NaN,GHCND:USC00055984,-9999.0,54,26,-9999,-9999,-9999,-9999,-9999,-9999


In [17]:
df.groupby(['DATE', 'STATION']).SNOW.count().value_counts()

1    338539
0     83495
2       169
Name: SNOW, dtype: int64

In [21]:
date_station_counts = df.groupby(['DATE', 'STATION']).SNOW.count().reset_index().rename(columns={'SNOW': 'counts'})

date_station_counts.head()

,DATE,STATION,counts
0,20000101,GHCND:USC00050848,1
1,20000101,GHCND:USC00051681,1
2,20000101,GHCND:USC00053629,1
3,20000101,GHCND:USC00055116,1
4,20000101,GHCND:USC00055984,1


In [22]:
date_station_counts[date_station_counts.counts == 2]

,DATE,STATION,counts
10224,20020101,GHCND:US1COBO0004,2
10225,20020101,GHCND:US1COBO0005,2
10226,20020101,GHCND:US1COBO0008,2
10227,20020101,GHCND:US1COBO0010,2
10228,20020101,GHCND:US1COBO0014,2
10229,20020101,GHCND:US1COBO0016,2
10230,20020101,GHCND:US1COBO0019,2
10231,20020101,GHCND:US1COBO0020,2
10232,20020101,GHCND:US1COBO0024,2
10233,20020101,GHCND:US1COBO0025,2


In [ ]:
# Oops, it looks like all the first days of years got duplicated.

In [24]:
df.drop_duplicates(subset=['DATE', 'STATION'], inplace=True)

In [25]:
df.groupby(['DATE', 'STATION']).SNOW.count().max()

1

In [10]:
df['DATE'] = pd.to_datetime(df.DATE, format='%Y%M%d')

In [26]:
df.to_csv('../data/boulder-weather-raw.csv', index=False)

In [29]:
df.head()

,DATE,PRCP,SNOW,SNWD,STATION,TAVG,TMAX,TMIN,WT01,WT03,WT04,WT05,WT06,WT11
328,20000101,0.00,0.0,NaN,GHCND:USC00050848,-9999.0,54,29,-9999,-9999,-9999,-9999,-9999,-9999
2949,20000101,0.06,1.3,NaN,GHCND:USC00051681,-9999.0,40,18,-9999,-9999,-9999,-9999,-9999,-9999
4608,20000101,0.00,0.0,NaN,GHCND:USC00053629,-9999.0,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999
6801,20000101,0.00,0.0,NaN,GHCND:USC00055116,-9999.0,57,21,-9999,-9999,-9999,-9999,-9999,-9999
8584,20000101,0.00,0.0,NaN,GHCND:USC00055984,-9999.0,54,26,-9999,-9999,-9999,-9999,-9999,-9999
